In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath('../')))

from tqdm import tqdm
import pandas as pd
from utils.file_utils import read_image, read_json, save_image, save_json, read_pkl
from utils.draw_utils import draw_box
from utils.helper_utils import float2_0_1000
from prompts import all_prompts

# make train

In [2]:
img_root = '/home/shaotao/DATA/os-altas/os-altas-linux'
ann_root = '/home/shaotao/PROJECTS/VLM_AND_PHONE/train_val_data_making/desktop/linux_patch_with_ocr_label.json'

ele_per_diag = 10
inst_type = 'func'
out_json_p = f'linux_patch_ocr_train.json'
prompt_type = 'ground_prompt_for_train'

In [ ]:
INIT_PROMPT = all_prompts[prompt_type]

CONTINUE_PROMPT = """## Instruction
{instruction}"""


df = pd.read_json(ann_root)
num_turn_per_diag = 10
import random
random.seed(42)
all_datas = []
for img_idx, (g_name, g_df) in enumerate(df.groupby('img_name')):
    img_p = os.path.join(img_root, g_name)
    if img_idx % 50 == 0:
        img = read_image(img_p)

    conversation = []
    count = 0
    for ele_idx, ele in g_df.iterrows():
        if count % num_turn_per_diag == 0 and count != 0:
            count = 0
            line = {'conversation': conversation, 'image_lst': [os.path.join(img_root, img_p)]}
            all_datas.append(line)
            conversation = []
            if len(all_datas) % 50 == 0:
                print(f'IDX: {len(all_datas)},  sample func_ann: {func_ann}')
            
        box = ele['box']
        func_ann = ele['model_pred'].strip()
        if 'Click to' in func_ann:
            func_ann = func_ann.replace('Click to', '').strip()
        if count == 0:   
            prompt = INIT_PROMPT.format(instruction=func_ann)
        else:
            prompt = CONTINUE_PROMPT.format(instruction=func_ann)
        count += 1
            
        x1, y1, x2, y2 = box
        cent_x, cent_y = (x1 + x2) / 2, (y1 + y2) / 2
        pt = [cent_x, cent_y]
        try:
            pt = list(map(float2_0_1000, pt))
        except Exception as e:
            print('idx: ', g_name, e)
            continue
        ans = f'({pt[0]},{pt[1]})'
        conversation.append({'from': 'human', 'value': prompt})
        conversation.append({'from': 'gpt', 'value': ans})
        if img_idx % 50 == 0:
            img = draw_box(img, (x1, y1, x2, y2))
            
    if img_idx % 50 == 0:
        save_image(img, f'tmp_{img_idx}.jpg')

print('total data num: ', len(all_datas))
save_json(all_datas, out_json_p)

In [ ]:
data1 = read_json('/home/shaotao/PROJECTS/VLM_AND_PHONE/train_val_data_making/desktop/linux_patch_ocr_train.json')
print('data1: ', len(data1))
data2 = read_json('/home/shaotao/PROJECTS/VLM_AND_PHONE/train_val_data_making/desktop/mac_patch_ocr_train.json')
print('data2: ', len(data2))
data1.extend(data2)
print('data1: ', len(data1))
save_json(data1, '/home/shaotao/PROJECTS/VLM_AND_PHONE/train_val_data_making/desktop/patch_ocr_train.json')